# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [179]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [180]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [181]:
data.head()

CustomerID FirstName   LastName  SalesID  ProductID           ProductName  \
0       61288      Rosa   Andersen   134196        229  Bread - Hot Dog Buns   
1       77352     Myron     Murray  6167892        229  Bread - Hot Dog Buns   
2       40094     Susan  Stevenson  5970885        229  Bread - Hot Dog Buns   
3       23548    Tricia    Vincent  6426954        229  Bread - Hot Dog Buns   
4       78981     Scott      Burch   819094        229  Bread - Hot Dog Buns   

   Quantity  
0        16  
1        20  
2        11  
3         6  
4        20

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [182]:
df1=pd.DataFrame()
df1["Quantity"]=data.copy().groupby(["CustomerID","ProductName"]).sum()["Quantity"]
df1.reset_index(inplace=True)
df1.head(3)

CustomerID              ProductName  Quantity
0          33         Apricots - Dried         1
1          33        Assorted Desserts         1
2          33  Bandage - Flexible Neon         1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [183]:
df2=pd.pivot_table(df1,values="Quantity",index='ProductName',columns=['CustomerID'],fill_value=0)
df2.head(3)

CustomerID                         33     200    264    356    412    464    \
ProductName                                                                   
Anchovy Paste - 56 G Tube              0      0      0      0      0      0   
Appetizer - Mini Egg Roll, Shrimp      0      0      0      0      0      0   
Appetizer - Mushroom Tart              0      0      0      0      0      0   

CustomerID                         477    639    649    669    ...  97697  \
ProductName                                                    ...          
Anchovy Paste - 56 G Tube              0      1      0      0  ...      0   
Appetizer - Mini Egg Roll, Shrimp      0      0      0      0  ...     25   
Appetizer - Mushroom Tart              0      1      0      0  ...     25   

CustomerID                         97753  97769  97793  97900  97928  98069  \
ProductName                                                                   
Anchovy Paste - 56 G Tube             25      0      0      0      0      0   
Appetizer - Mini Egg Roll, Shrimp     25      0      0      0      0      0   
Appetizer - Mushroom Tart              0      0      0      0      0      0   

CustomerID                         98159  98185  98200  
ProductName                                             
Anchovy Paste - 56 G Tube              0      0      0  
Appetizer - Mini Egg Roll, Shrimp      0      0      0  
Appetizer - Mushroom Tart              0     25      0  

[3 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [184]:
df3 = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'euclidean'))), 
                         index=df2.columns, columns=df2.columns)

df3.head(5)

CustomerID     33        200       264       356       412       464    \
CustomerID                                                               
33          1.000000  0.077421  0.087047  0.081800  0.080634  0.082709   
200         0.077421  1.000000  0.078448  0.076435  0.073693  0.075255   
264         0.087047  0.078448  1.000000  0.080070  0.081800  0.080350   
356         0.081800  0.076435  0.080070  1.000000  0.076435  0.078187   
412         0.080634  0.073693  0.081800  0.076435  1.000000  0.078711   

CustomerID     477       639       649       669    ...     97697     97753  \
CustomerID                                          ...                       
33          0.074573  0.083020  0.081503  0.080070  ...  0.004811  0.004669   
200         0.075956  0.076435  0.077674  0.076923  ...  0.004824  0.004681   
264         0.076923  0.080634  0.082100  0.078448  ...  0.004822  0.004674   
356         0.075025  0.082403  0.077171  0.075956  ...  0.004816  0.004671   
412         0.075025  0.082403  0.078187  0.078448  ...  0.004810  0.004702   

CustomerID     97769     97793     97900     97928     98069     98159  \
CustomerID                                                               
33          0.004412  0.005019  0.004312  0.004515  0.004583  0.004355   
200         0.004431  0.005047  0.004311  0.004521  0.004614  0.004367   
264         0.004416  0.005035  0.004322  0.004543  0.004595  0.004365   
356         0.004416  0.005038  0.004310  0.004526  0.004578  0.004365   
412         0.004414  0.005034  0.004318  0.004530  0.004578  0.004367   

CustomerID     98185     98200  
CustomerID                      
33          0.004167  0.004333  
200         0.004166  0.004335  
264         0.004179  0.004333  
356         0.004175  0.004339  
412         0.004177  0.004349  

[5 rows x 1000 columns]

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [185]:
df4=df3.copy().reset_index()[["CustomerID",33]].sort_values(33,ascending=False)
df4=df4[1:].head(5)
df4

CustomerID  CustomerID        33
44                3317  0.087047
47                3535  0.087047
2                  264  0.087047
29                2503  0.085983
43                3305  0.085638

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [186]:
lista1=list(df4["CustomerID"])
df5=df1[df1["CustomerID"].isin(lista)]
df5.head(3)

CustomerID                 ProductName  Quantity
131         264           Apricots - Halves         1
132         264              Apricots Fresh         1
133         264  Bacardi Breezer - Tropical         1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [187]:
df6=pd.DataFrame()
df6["Quantity"]=df5.copy().groupby(["ProductName"]).sum()["Quantity"]
df6.reset_index(inplace=True)
df6.sort_values('Quantity',ascending=False,inplace=True)
df6.head(5)

ProductName  Quantity
33               Butter - Unsalted         3
203  Wine - Ej Gallo Sierra Valley         3
186         Towels - Paper / Kraft         3
168   Soup - Campbells Bean Medley         3
199  Wine - Blue Nun Qualitatswein         3

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [188]:
df7_0=df6.copy().merge(df2,on="ProductName")
df7_0=df7_0.copy()[df7_0[33]==0][["ProductName","Quantity"]]
df7_0=df7_0.copy().head()
df7_0

ProductName  Quantity
0              Butter - Unsalted         3
1  Wine - Ej Gallo Sierra Valley         3
3   Soup - Campbells Bean Medley         3
4  Wine - Blue Nun Qualitatswein         3
6            Chicken - Soup Base         2

In [189]:
#Different approach:
lista2=(products[products["CustomerID"]==33]["ProductName"])
df7_1=df6.copy()[df6['ProductName'].isin(lista2)==False].sort_values('Quantity',ascending=False)
df7_1=df7_1.copy().head()
df7_1

ProductName  Quantity
33               Butter - Unsalted         3
168   Soup - Campbells Bean Medley         3
199  Wine - Blue Nun Qualitatswein         3
203  Wine - Ej Gallo Sierra Valley         3
14                     Beans - Wax         2

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [190]:
recom={}
cust_ids=list(df1.copy()["CustomerID"].unique())
def recommend(_id):
    #step 4:
    ddf4=df3.copy().reset_index()[["CustomerID",_id]].sort_values(_id,ascending=False)
    ddf4=ddf4[1:].head(5)
    #step 5:
    cust_list=list(ddf4["CustomerID"])
    ddf5=df1[df1["CustomerID"].isin(cust_list)]
    #step 6:
    ddf6=pd.DataFrame()
    ddf6["Quantity"]=ddf5.copy().groupby(["ProductName"]).sum()["Quantity"]
    ddf6.reset_index(inplace=True)
    ddf6.sort_values('Quantity',ascending=False,inplace=True)
    #step 7:
    ddf7=ddf6.copy().merge(df2,on="ProductName")
    ddf7=ddf7.copy()[ddf7[_id]==0]["ProductName"]
    rec_list=list(ddf7[:5])
    return rec_list

recom={i:recommend(i) for i in cust_ids}
   

In [192]:
recom[264]

['Soupfoamcont12oz 112con',
 'Wine - Two Oceans Cabernet',
 'Bread - Italian Roll With Herbs',
 'Veal - Inside, Choice',
 'Fish - Scallops, Cold Smoked']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [193]:
customer_rec=pd.DataFrame(recom).T
customer_rec.reset_index(inplace=True)
customer_rec.columns=["CustomerID","Recomendation 1","Recomendation 2","Recomendation 3","Recomendation 4","Recomendation 5"]
customer_rec.head()

CustomerID               Recomendation 1                  Recomendation 2  \
0          33             Butter - Unsalted    Wine - Ej Gallo Sierra Valley   
1         200  Soup - Campbells Bean Medley  Muffin - Carrot Individual Wrap   
2         264       Soupfoamcont12oz 112con       Wine - Two Oceans Cabernet   
3         356             Butter - Unsalted            Veal - Inside, Choice   
4         412       Olive - Spread Tapenade      Sprouts - Baby Pea Tendrils   

                   Recomendation 3                Recomendation 4  \
0     Soup - Campbells Bean Medley  Wine - Blue Nun Qualitatswein   
1                         Bay Leaf                  Pork - Kidney   
2  Bread - Italian Roll With Herbs          Veal - Inside, Choice   
3      Beets - Candy Cane, Organic         Nut - Chestnuts, Whole   
4    Wine - Blue Nun Qualitatswein          Pepper - Black, Whole   

                Recomendation 5  
0           Chicken - Soup Base  
1                   Wanton Wrap  
2  Fish - Scallops, Cold Smoked  
3                 Lamb - Ground  
4  Soup - Campbells Bean Medley

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [194]:
df3_new = pd.DataFrame(1/(1 + squareform(pdist(df2.T, 'cosine'))), index=df2.columns, columns=df2.columns)

In [195]:
recom_n={}
cust_ids=list(df1.copy()["CustomerID"].unique())
def recommend_n(_id):
    #step 4:
    ddf4=df3_new.copy().reset_index()[["CustomerID",_id]].sort_values(_id,ascending=False)
    ddf4=ddf4[1:].head(5)
    #step 5:
    cust_list=list(ddf4["CustomerID"])
    ddf5=df1[df1["CustomerID"].isin(cust_list)]
    #step 6:
    ddf6=pd.DataFrame()
    ddf6["Quantity"]=ddf5.copy().groupby(["ProductName"]).sum()["Quantity"]
    ddf6.reset_index(inplace=True)
    ddf6.sort_values('Quantity',ascending=False,inplace=True)
    #step 7:
    ddf7=ddf6.copy().merge(df2,on="ProductName")
    ddf7=ddf7.copy()[ddf7[_id]==0]["ProductName"]
    rec_list=list(ddf7[:5])
    return rec_list

recom_n={i:recommend_n(i) for i in cust_ids}

In [200]:
customer_rec_n=pd.DataFrame(recom_n).T
customer_rec_n.reset_index(inplace=True)
customer_rec_n.columns=["CustomerID","Recomendation 1","Recomendation 2","Recomendation 3","Recomendation 4","Recomendation 5"]
customer_rec_n.head()

CustomerID                   Recomendation 1  \
0          33             Knife Plastic - White   
1         200  Longos - Grilled Salmon With Bbq   
2         264                Pickerel - Fillets   
3         356            Bread - English Muffin   
4         412                  Salsify, Organic   

                 Recomendation 2                Recomendation 3  \
0  Soup - Campbells, Beef Barley             Onions - Cippolini   
1              Snapple Lemon Tea        General Purpose Trigger   
2       Water - Mineral, Natural       Snapple - Iced Tea Peach   
3        Olive - Spread Tapenade                  Bagel - Plain   
4                   Durian Fruit  Wine - Hardys Bankside Shiraz   

                 Recomendation 4                 Recomendation 5  
0      Tea - Herbal Sweet Dreams          Muffin - Zero Transfat  
1           Thyme - Lemon, Fresh              Tomatoes Tear Drop  
2  Wine - Ej Gallo Sierra Valley  French Pastry - Mini Chocolate  
3  Pork - Hock And Feet Attached               Wiberg Super Cure  
4     Bread - Raisin Walnut Oval        Gatorade - Xfactor Berry

In [201]:
#and these are the previous recommendations (euclidean):
customer_rec.head()

CustomerID               Recomendation 1                  Recomendation 2  \
0          33             Butter - Unsalted    Wine - Ej Gallo Sierra Valley   
1         200  Soup - Campbells Bean Medley  Muffin - Carrot Individual Wrap   
2         264       Soupfoamcont12oz 112con       Wine - Two Oceans Cabernet   
3         356             Butter - Unsalted            Veal - Inside, Choice   
4         412       Olive - Spread Tapenade      Sprouts - Baby Pea Tendrils   

                   Recomendation 3                Recomendation 4  \
0     Soup - Campbells Bean Medley  Wine - Blue Nun Qualitatswein   
1                         Bay Leaf                  Pork - Kidney   
2  Bread - Italian Roll With Herbs          Veal - Inside, Choice   
3      Beets - Candy Cane, Organic         Nut - Chestnuts, Whole   
4    Wine - Blue Nun Qualitatswein          Pepper - Black, Whole   

                Recomendation 5  
0           Chicken - Soup Base  
1                   Wanton Wrap  
2  Fish - Scallops, Cold Smoked  
3                 Lamb - Ground  
4  Soup - Campbells Bean Medley